**UNIVERSIDAD GALILEO**

**Marco Vinicio Escalante Lara**

Carnet: **19001148**

# Proyecto Statistical Learning I


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import sklearn.metrics as mt
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
import tensorflow as tf

## Cargar datos y descripción de información
Para ver con que información contamos realizamos una muestra de los primeros diez registros de la tabla. Así como también trabajamos una descripción de las variables, para saber su distribución entre otros e iniciar el trabajo con nuestras variables.

In [2]:
Titanic = pd.read_csv("titanic.csv")
Titanic.head(10) ,Titanic.describe()

(   PassengerId                                               Name   Age  \
 0            1                            Braund, Mr. Owen Harris  22.0   
 1            2  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0   
 2            3                             Heikkinen, Miss. Laina  26.0   
 3            4       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0   
 4            5                           Allen, Mr. William Henry  35.0   
 5            6                                   Moran, Mr. James   NaN   
 6            7                            McCarthy, Mr. Timothy J  54.0   
 7            8                     Palsson, Master. Gosta Leonard   2.0   
 8            9  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)  27.0   
 9           10                Nasser, Mrs. Nicholas (Adele Achem)  14.0   
 
    SibSp  Parch            Ticket     Fare Cabin Embarked passenger_class  \
 0      1      0         A/5 21171   7.2500   NaN        S           Lower   
 1    

## Selección de variables y limpieza

### Por el tipo de datos que tenemos vamos a trabajar con los siguientes (One hot encoding):

**Edad** --> Es un número discreto, si tiene NA o NULL le colocaremos la mediana de los datos.

**SibSP** --> Es un número discreto, que indica la cantidad de hermanos/esposas que estaban a bordo por pasajero.

**Parch** --> Es un número discreto, que indica la cantidad de padres/hijos que estaban a bordo por pasajero.

**Fare** --> Es un número continuo, que indica la tarifa que pago el pasajero.

**Embarked** --> Es una letra, que se recodifico para tener una variable númerica, donde 1 significa puerta de embarque Cherbourg, 2 signficca puerta de embarque Queenstown y 3 puerta de embarque Southampton.

**passenger_class** --> Es una palabra que indica en que clase viajan los pasajeros, 1 significa primera clase, 2 significa segunda clase y 3 tercera clase.

**passenger_sex** --> Es una letra, que indica el sexo del pasajero, la cual reclasificamos en 0 para hombres y 1 para mujeres.

La variable que queremos clasificar:

**passenger_survived** --> Es una letra que indica si el pasajero sobrevivio o no, la cual reclasificamos con 1 para sobrevivio y 0 para no sobrevivio.   

### Debido a que trabajamos Feature Engeenier no tomamos, las siguientes variables:

**PassengerId** --> Ya que solo identifica a los pasajeros.

**Ticket** --> Es una combinación numerica del ticket del pasajero

**Name** --> Es el nombre del pasajero.

**Cabin** --> Es el número de la cabina donde se hospedo el pasajero.


In [4]:
# Colocar la mediana a los NA de edad
Titanic["Age"] = Titanic["Age"].fillna(Titanic["Age"].median());
Titanic["Age"].isnull().sum()
# Cambiar de letra a numero el sexo
Titanic["Sexo"] = (Titanic["passenger_sex"] == 'F').astype(np.float)
# Definiendo las variables
target = ["passenger_survived"]
numeric_fields = ["Age", "SibSp", "Parch", "Fare", "Sexo", "Embarked", "passenger_class"]
y = (Titanic[target].values == 'Y').astype(np.float)
# One hot encoding, cambiando variables categoricas a dummies
X =  pd.get_dummies(Titanic[numeric_fields])

# Muestra de la descripción de datos de nuestras variables a utilizar
X.head(5)

,Age,SibSp,Parch,Fare,Sexo,Embarked_C,Embarked_Q,Embarked_S,passenger_class_Lower,passenger_class_Middle,passenger_class_Upper
0,22.0,1,0,7.2500,0.0,0,0,1,1,0,0
1,38.0,1,0,71.2833,1.0,1,0,0,0,0,1
2,26.0,0,0,7.9250,1.0,0,0,1,1,0,0
3,35.0,1,0,53.1000,1.0,0,0,1,0,0,1
4,35.0,0,0,8.0500,0.0,0,0,1,1,0,0


## Separación de entreno, validación y prueba

Por medio de Sklearn, como sugerencia en el proyecto.

In [5]:
# Entrenamiento, validación y de prueba 
X_entreno1, X_test, y_entreno1, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42)
# Separación de los datos 
X_entreno, X_val, y_entreno, y_val = train_test_split(X_entreno1, y_entreno1, test_size=0.2, random_state=42)
# Dimensiones de la muestra
X_entreno.shape, X_val.shape, X_test.shape

((569, 11), (143, 11), (179, 11))

## Árbol de decisión

Arboles de decisión tomando los datos que trabajos anteriormente

In [6]:
def arbol_segmentacion(X_entreno, X_val, y_entreno, y_val, parametro): 
    arbol = tree.DecisionTreeClassifier(max_depth = parametro)
    arbol = arbol.fit(X_entreno, y_entreno)
    arbol.fit(X_entreno,y_entreno)
    # Medidas de entreno y validación obtenidas por Sklearn
    # Medidas de entreno
    y_pred_e = arbol.predict(X_entreno)
    y_pred_v = arbol.predict(X_val)
    medidas_entreno = [mt.accuracy_score(y_entreno, y_pred_e, normalize=True), mt.f1_score(y_entreno, y_pred_e), 
                       mt.precision_score(y_entreno, y_pred_e), 
                       mt.recall_score(y_entreno, y_pred_e, average='weighted')]
    medidas_validacion = [mt.accuracy_score(y_val, y_pred_v, normalize=True), mt.f1_score(y_val, y_pred_v), 
                          mt.precision_score(y_val, y_pred_v), 
                          mt.recall_score(y_val, y_pred_v, average='weighted')]
    # Retorna el modelo y las medidas de entreno y validación
    return arbol, medidas_entreno, medidas_validacion


In [24]:
metricas_entreno = []
metricas_validacion = []
profundidad_lista = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20]
for n in profundidad_lista: 
    _, metrica_e, metrica_v = arbol_segmentacion(X_entreno, X_val, y_entreno, y_val, n)
    metricas_entreno.append(metrica_e)
    metricas_validacion.append(metrica_v)

## Resultados de las Metricas del Árbol

A continuación se muestran los resultados de las metricas para el modelo de árboles de segmentación:

In [25]:
resultado_arbol = pd.DataFrame(np.column_stack((profundidad_lista, np.array(metricas_entreno), 
                                                np.array(metricas_validacion))), 
                               columns = ['Profundidad', 'Accuracy_Entreno', 'f1_Entreno', 'Precision_Entreno', 
                                          'Recall_Entreno', 'Accuracy_Val', 'f1_Val', 'Precision_Val', 
                                          'Recall_Val'])
resultado_arbol

,Profundidad,Accuracy_Entreno,f1_Entreno,Precision_Entreno,Recall_Entreno,Accuracy_Val,f1_Val,Precision_Val,Recall_Val
0,2.0,0.799649,0.650307,0.929825,0.799649,0.818182,0.723404,0.894737,0.818182
1,3.0,0.833040,0.760705,0.816216,0.833040,0.832168,0.773585,0.820000,0.832168
2,4.0,0.838313,0.770000,0.819149,0.838313,0.839161,0.785047,0.823529,0.839161
3,5.0,0.855888,0.779570,0.906250,0.855888,0.825175,0.752475,0.844444,0.825175
4,6.0,0.880492,0.817204,0.950000,0.880492,0.790210,0.705882,0.782609,0.790210
5,7.0,0.892794,0.839895,0.946746,0.892794,0.825175,0.757282,0.829787,0.825175
6,8.0,0.906854,0.860158,0.976048,0.906854,0.797203,0.712871,0.800000,0.797203
7,9.0,0.919156,0.878947,0.994048,0.919156,0.818182,0.754717,0.800000,0.818182
8,10.0,0.926186,0.891192,0.988506,0.926186,0.797203,0.728972,0.764706,0.797203
9,20.0,0.977153,0.968974,0.980676,0.977153,0.790210,0.732143,0.732143,0.790210


## Supor Vector Machine

SVM tomando los datos que trabajos anteriormente, vamos a trabajar con un kernel Gaussiano

In [46]:
def svm_funcion(X_entreno, X_val, y_entreno, y_val, C_param): 
    svm_model = Pipeline([("scaler", StandardScaler()), 
                          ("svc", SVC(C = C_param, kernel='rbf', tol = 0.001, max_iter = 5000))])
    svm_model.fit(X_entreno, y_entreno.reshape(len(y_entreno), ))
    # Medidas de entreno y validación obtenidas por Sklearn
    # Medidas de entreno
    y_pred_e = svm_model.predict(X_entreno)
    y_pred_v = svm_model.predict(X_val)
    medidas_entreno = [mt.accuracy_score(y_entreno, y_pred_e, normalize=True), 
                       mt.f1_score(y_entreno, y_pred_e), mt.precision_score(y_entreno, y_pred_e), 
                       mt.recall_score(y_entreno, y_pred_e, average='weighted')]
    medidas_validacion = [mt.accuracy_score(y_val, y_pred_v, normalize=True), 
                          mt.f1_score(y_val, y_pred_v),  mt.precision_score(y_val, y_pred_v), 
                          mt.recall_score(y_val, y_pred_v, average='weighted')]
    # Retorna el modelo y las medidas de entreno y validación
    return svm_model, medidas_entreno, medidas_validacion

In [47]:
parametro = [0.1, 1., 2., 3., 4., 5., 6., 7., 10., 15.]
metricas_entreno = []
metricas_validacion = []
for n in parametro:
    _, metrica_e, metrica_v = svm_funcion(X_entreno, X_val, y_entreno, y_val, n)
    metricas_entreno.append(metrica_e.copy())
    metricas_validacion.append(metrica_v.copy())

In [48]:
Resultados_SVM = pd.DataFrame(np.column_stack((parametro, np.array(metricas_entreno), 
                                           np.array(metricas_validacion))), 
                          columns = ['Parametro',  'Accuracy_Entreno', 'f1_Entreno', 'Precision_Entreno', 
                                     'Recall_Entreno', 'Accuracy_Val', 'f1_Val', 'Precision_Val', 
                                     'Recall_Val'])
Resultados_SVM

,Parametro,Accuracy_Entreno,f1_Entreno,Precision_Entreno,Recall_Entreno,Accuracy_Val,f1_Val,Precision_Val,Recall_Val
0,0.1,0.808436,0.702997,0.832258,0.808436,0.811189,0.737864,0.808511,0.811189
1,1.0,0.843585,0.756164,0.901961,0.843585,0.839161,0.772277,0.866667,0.839161
2,2.0,0.843585,0.754821,0.907285,0.843585,0.846154,0.780000,0.886364,0.846154
3,3.0,0.847100,0.761644,0.908497,0.847100,0.839161,0.772277,0.866667,0.839161
4,4.0,0.848858,0.763736,0.914474,0.848858,0.839161,0.772277,0.866667,0.839161
5,5.0,0.850615,0.767123,0.915033,0.850615,0.839161,0.772277,0.866667,0.839161
6,6.0,0.854130,0.776280,0.905660,0.854130,0.839161,0.772277,0.866667,0.839161
7,7.0,0.859402,0.786096,0.907407,0.859402,0.832168,0.764706,0.847826,0.832168
8,10.0,0.871705,0.808399,0.911243,0.871705,0.811189,0.742857,0.795918,0.811189
9,15.0,0.869947,0.806283,0.905882,0.869947,0.804196,0.730769,0.791667,0.804196


## Naive Bayes

Naive Bayes tomando los datos que trabajos anteriormente

In [12]:
def naive_bayes(X_entreno, X_val, y_entreno, y_val, variables):
    #Se trabaja con un clasificador Gausiano, especifico para estos modelos
    modelo_NB = GaussianNB()
    
    modelo_NB.fit(X_entreno.values[:, variables], y_entreno.reshape(len(y_entreno),))
    # Medidas de entreno y validación obtenidas por Sklearn
    # Medidas de entreno
    y_pred_e = modelo_NB.predict(X_entreno.values[:, variables])
    y_pred_v = modelo_NB.predict(X_val.values[:, variables])
    
    
    medidas_entreno = [mt.accuracy_score(y_entreno, y_pred_e, normalize=True), mt.f1_score(y_entreno, y_pred_e), 
                       mt.precision_score(y_entreno, y_pred_e), 
                       mt.recall_score(y_entreno, y_pred_e, average='weighted')]
    #Medidas de validación
    medidas_validacion = [mt.accuracy_score(y_val, y_pred_v, normalize=True), mt.f1_score(y_val, y_pred_v), 
                          mt.precision_score(y_val, y_pred_v), 
                          mt.recall_score(y_val, y_pred_v, average='weighted')]
    
    return modelo_NB, medidas_entreno, medidas_validacion


In [20]:
metricas_entreno = []
metricas_validacion = []
lista1=[[0,1,2,3,4,5],[0,1,2,3,4,5,6],
        [1,2,3,4,5,6,7,8], [0,1,3,5,6,8,9,10],[1,3,4,5,6,7,8,9],[0,1,4,5,6,8,9,10],[0,1,2,4,6,7,8,10],
        [1,2,3,4,5,7,8,9,10], [0,1,3,5,6,7,8,9,10], [1,2,3,5,6,7,8,9,10]]

for i in lista1: 
    _, metrica_e, metrica_v = naive_bayes(X_entreno, X_val, y_entreno, y_val, i)
    metricas_entreno.append(metrica_e)
    metricas_validacion.append(metrica_v)

In [21]:
NaiveBayes_resultado = pd.DataFrame(np.column_stack((lista1, np.array(metricas_entreno), 
                                                     np.array(metricas_validacion))), 
                             columns = ['Variables', 'Accuracy_Entreno', 'f1_Entreno', 'Precision_Entreno', 
                                     'Recall_Entreno', 'Accuracy_Val', 'f1_Val', 'Precision_Val', 
                                     'Recall_Val'])
NaiveBayes_resultado

,Variables,Accuracy_Entreno,f1_Entreno,Precision_Entreno,Recall_Entreno,Accuracy_Val,f1_Val,Precision_Val,Recall_Val
0,"[0, 1, 2, 3, 4, 5]",0.810193,0.732673,0.770833,0.810193,0.797203,0.743363,0.736842,0.797203
1,"[0, 1, 2, 3, 4, 5, 6]",0.811951,0.735802,0.772021,0.811951,0.783217,0.725664,0.719298,0.783217
2,"[1, 2, 3, 4, 5, 6, 7, 8]",0.796134,0.718447,0.74,0.796134,0.79021,0.745763,0.709677,0.79021
3,"[0, 1, 3, 5, 6, 8, 9, 10]",0.70123,0.538043,0.634615,0.70123,0.643357,0.49505,0.555556,0.643357
4,"[1, 3, 4, 5, 6, 7, 8, 9]",0.787346,0.679045,0.775758,0.787346,0.804196,0.745455,0.759259,0.804196
5,"[0, 1, 4, 5, 6, 8, 9, 10]",0.766257,0.715203,0.654902,0.766257,0.72028,0.69697,0.605263,0.72028
6,"[0, 1, 2, 4, 6, 7, 8, 10]",0.780316,0.724062,0.680498,0.780316,0.72028,0.692308,0.608108,0.72028
7,"[1, 2, 3, 4, 5, 7, 8, 9, 10]",0.766257,0.679518,0.694581,0.766257,0.755245,0.710744,0.661538,0.755245
8,"[0, 1, 3, 5, 6, 7, 8, 9, 10]",0.697715,0.537634,0.625,0.697715,0.615385,0.444444,0.511628,0.615385
9,"[1, 2, 3, 5, 6, 7, 8, 9, 10]",0.667838,0.509091,0.566474,0.667838,0.622378,0.470588,0.521739,0.622378


## Regresión Logistica

Regresión Logistica tomando los datos que trabajos anteriormente

In [16]:
def regresion(X_entreno, X_val, y_entreno, y_val, lr, lambda_val, epochs, tag):
    
    # Definimos el tamaño de nuestras variables
    w_ =[]
    m, k = X_entreno.shape
    tf.reset_default_graph()
    gra = tf.Graph()
    # Trabajamos el cascaron
    with gra.as_default():
        # Creando los placeholders que reciben las variables
        X = tf.placeholder(tf.float32, shape = (None, k), name = "X")
        Ylabels = tf.placeholder(tf.float32, name = "Variables_y")

        # Tenemos el Learning Rate y Lambda como parametros del modelo (Hiperparametros)
        lr_param = tf.placeholder(tf.float32, name = "lr")
        lambda_param = tf.placeholder(tf.float32, name = "lambda")

        # Los pesos de la regresión
        # Coeficientes
        W = tf.Variable(tf.truncated_normal(shape = [k, 1]), name = "W")
        # Intercepto
        b = tf.Variable(tf.truncated_normal(shape = (1, 1)), name = "b")
        
        # Definimos la regresión logistica
        with tf.name_scope("Regresion"):
            Logits = tf.add(tf.matmul(X, W), b, name = "Regresion")
            YlabelsHat = tf.nn.sigmoid(Logits)

        # Función de costo
        with tf.name_scope("Funcion_Costo"):
            w_norm = tf.divide(tf.multiply(tf.multiply(tf.constant(0.5), lambda_param), 
                                           tf.reduce_sum(tf.square(W))), tf.cast(m, tf.float32), 
                               name = "W_norm")
            # Entropia cruzada
            classif_term = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                labels = Ylabels, logits = Logits), name = "CostoClasif") 
            # Costo final, sumar la función de costo con la norma de los pesos
            cost = tf.add(classif_term, w_norm, name="Costo")

        # Gradient Descent Optimizer - Solicitado en el proyecto
        with tf.name_scope("Gradient_Descent_Optimizer"):
            optimizer = tf.train.GradientDescentOptimizer(lr_param).minimize(cost) 
        
        # Inicializador de variables globales
        init = tf.global_variables_initializer() 

    with tf.Session(graph = gra) as sess:    
        sess.run(init)        
        for epoch in range(epochs):
            # Corriendo los batch
            _, c_ = sess.run([optimizer, cost], feed_dict = {X : X_entreno, Ylabels : y_entreno.reshape((m, 1)), 
                                                             lr_param : lr, lambda_param : lambda_val})
            
            if (epoch + 1) % 100 == 0:
                print("Epoch: %d, \t costo = %0.4f" % (epoch+1, c_))
            #Parametros finales, coeficientes e intercepto
            w_, b_ = sess.run([W, b])
                
        # Devolver los parámetros
        return w_, b_,

In [17]:
def metrica_regresion(w, b, X_entreno, y_entreno, X_val, y_val):
    
    # Predicción binaria
    y_pred_e = 1.0*((1 / (1 + np.exp(-(np.matmul(X_entreno.values,w)+b)))) > 0.5)
        
    medidas_entreno = [mt.accuracy_score(y_entreno, y_pred_e, normalize=True), 
                           mt.f1_score(y_entreno, y_pred_e),mt.precision_score(y_entreno, y_pred_e), 
                           mt.recall_score(y_entreno, y_pred_e, average='weighted')]
    
    # Métricas en datos de entrenamiento
    y_pred_v = 1.0*((1 / (1 + np.exp(-(np.matmul(X_val.values,w)+b)))) > 0.5)
    medidas_validacion = [mt.accuracy_score(y_val, y_pred_v, normalize=True), 
                              mt.f1_score(y_val, y_pred_v), mt.precision_score(y_val, y_pred_v), 
                              mt.recall_score(y_val, y_pred_v, average='weighted')]
    return medidas_entreno, medidas_validacion

In [18]:

Parmetros_lam =[0.1,0.5,1.,2.,3.,4.,5.,10.,15.,20.]
    
metricas_entreno = []
metricas_validacion = []
w_, b_ = regresion(X_entreno, X_val, y_entreno, y_val, 0.001, 0.1, 1000, '')
metrica_e, metrica_v = metrica_regresion(w_, b_, X_entreno, y_entreno, X_val, y_val)
metricas_entreno.append(metrica_e)
metricas_validacion.append(metrica_v)

w_, b_ = regresion(X_entreno, X_val, y_entreno, y_val, 0.001, 0.5, 1000, '')
metrica_e, metrica_v = metrica_regresion(w_, b_, X_entreno, y_entreno, X_val, y_val)
metricas_entreno.append(metrica_e)
metricas_validacion.append(metrica_v)

w_, b_ = regresion(X_entreno, X_val, y_entreno, y_val, 0.001, 1., 1000, '')
metrica_e, metrica_v = metrica_regresion(w_, b_, X_entreno, y_entreno, X_val, y_val)
metricas_entreno.append(metrica_e)
metricas_validacion.append(metrica_v)

w_, b_ = regresion(X_entreno, X_val, y_entreno, y_val, 0.001, 2., 1000, '')
metrica_e, metrica_v = metrica_regresion(w_, b_, X_entreno, y_entreno, X_val, y_val)
metricas_entreno.append(metrica_e)
metricas_validacion.append(metrica_v)

w_, b_ = regresion(X_entreno, X_val, y_entreno, y_val, 0.001, 3., 1000, '')
metrica_e, metrica_v = metrica_regresion(w_, b_, X_entreno, y_entreno, X_val, y_val)
metricas_entreno.append(metrica_e)
metricas_validacion.append(metrica_v)

w_, b_ = regresion(X_entreno, X_val, y_entreno, y_val, 0.001, 4., 1000, '')
metrica_e, metrica_v = metrica_regresion(w_, b_, X_entreno, y_entreno, X_val, y_val)
metricas_entreno.append(metrica_e)
metricas_validacion.append(metrica_v)

w_, b_ = regresion(X_entreno, X_val, y_entreno, y_val, 0.001, 5., 1000, '')
metrica_e, metrica_v = metrica_regresion(w_, b_, X_entreno, y_entreno, X_val, y_val)
metricas_entreno.append(metrica_e)
metricas_validacion.append(metrica_v)

w_, b_ = regresion(X_entreno, X_val, y_entreno, y_val, 0.001, 10., 1000, '')
metrica_e, metrica_v = metrica_regresion(w_, b_, X_entreno, y_entreno, X_val, y_val)
metricas_entreno.append(metrica_e)
metricas_validacion.append(metrica_v)

w_, b_ = regresion(X_entreno, X_val, y_entreno, y_val, 0.001, 15., 1000, '')
metrica_e, metrica_v = metrica_regresion(w_, b_, X_entreno, y_entreno, X_val, y_val)
metricas_entreno.append(metrica_e)
metricas_validacion.append(metrica_v)

w_, b_ = regresion(X_entreno, X_val, y_entreno, y_val, 0.001, 20., 1000, '')
metrica_e, metrica_v = metrica_regresion(w_, b_, X_entreno, y_entreno, X_val, y_val)
metricas_entreno.append(metrica_e)
metricas_validacion.append(metrica_v)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch: 100, 	 costo = 0.8112
Epoch: 200, 	 costo = 0.8072
Epoch: 300, 	 costo = 0.8035
Epoch: 400, 	 costo = 0.7999
Epoch: 500, 	 costo = 0.7963
Epoch: 600, 	 costo = 0.7927
Epoch: 700, 	 costo = 0.7892
Epoch: 800, 	 costo = 0.7857
Epoch: 900, 	 costo = 0.7822
Epoch: 1000, 	 costo = 0.7788
Epoch: 100, 	 costo = 0.9313
Epoch: 200, 	 costo = 0.9273
Epoch: 300, 	 costo = 0.9235
Epoch: 400, 	 costo = 0.9197
Epoch: 500, 	 costo = 0.9160
Epoch: 600, 	 costo = 0.9123
Epoch: 700, 	 costo = 0.9086
Epoch: 800, 	 costo = 0.9050
Epoch: 900, 	 costo = 0.9015
Epoch: 1000, 	 costo = 0.8979
Epoch: 100, 	 costo = 0.6636
Epoch: 200, 	 costo = 0.6572
Epoch: 300, 	 costo = 0.6553
Epoch: 400, 	 costo = 0.6534
Epoch: 500, 	 costo = 0.6515
Epoch: 600, 	 costo = 0.6497
Epoch: 700, 	 costo = 0.6479
Epoch: 800, 	 costo = 0.6461
Epoch: 900, 	 costo = 0.6443
Epoch: 1000, 	 costo = 0.6426
Epoch: 

In [19]:
Regresion_resultado = pd.DataFrame(np.column_stack((Parmetros_lam, np.array(metricas_entreno), 
                                                     np.array(metricas_validacion))), 
                             columns = ['Lambda', 'Accuracy_Entreno', 'f1_Entreno', 'Precision_Entreno', 
                                     'Recall_Entreno', 'Accuracy_Val', 'f1_Val', 'Precision_Val', 
                                     'Recall_Val'])
Regresion_resultado

,Lambda,Accuracy_Entreno,f1_Entreno,Precision_Entreno,Recall_Entreno,Accuracy_Val,f1_Val,Precision_Val,Recall_Val
0,0.1,0.585237,0.415842,0.437500,0.585237,0.594406,0.408163,0.476190,0.594406
1,0.5,0.616872,0.377143,0.478261,0.616872,0.538462,0.266667,0.352941,0.538462
2,1.0,0.671353,0.547215,0.562189,0.671353,0.734266,0.666667,0.655172,0.734266
3,2.0,0.643234,0.382979,0.538462,0.643234,0.573427,0.298851,0.419355,0.573427
4,3.0,0.579965,0.415648,0.431472,0.579965,0.580420,0.347826,0.444444,0.580420
5,4.0,0.690685,0.531915,0.609756,0.690685,0.685315,0.516129,0.648649,0.685315
6,5.0,0.604569,0.489796,0.471616,0.604569,0.671329,0.552381,0.591837,0.671329
7,10.0,0.706503,0.532213,0.655172,0.706503,0.622378,0.413043,0.527778,0.622378
8,15.0,0.659051,0.535885,0.543689,0.659051,0.650350,0.519231,0.562500,0.650350
9,20.0,0.687170,0.500000,0.618056,0.687170,0.713287,0.559140,0.702703,0.713287


## Ensambling Learning

A continuación trabajaremos uniendo todos los modelos previamente trabajados en uno solo, para tomar la mejor desición, para realizar esto se toma una moda de los ejercicios que trabajamos, en este caso toma la moda de los 4 modelos planteados.

In [43]:
pd.DataFrame(np.column_stack((Regresion_resultado['Lambda'],Regresion_resultado['Accuracy_Entreno'],
                              Regresion_resultado['Accuracy_Val'],NaiveBayes_resultado['Variables'],
                             NaiveBayes_resultado['Accuracy_Entreno'],NaiveBayes_resultado['Accuracy_Val'],
                             Resultados_SVM['Parametro'],Resultados_SVM['Accuracy_Entreno'],
                              Resultados_SVM['Accuracy_Val'],resultado_arbol['Profundidad'],
                             resultado_arbol['Accuracy_Entreno'],resultado_arbol['Accuracy_Val'])),
            columns=['Lambda_Reg', 'Accuracy_Ent_Regresion', 'Accuracy_Val_Regresion','Variables_Bayes',
                     'Accuracy_Ent_Bayes', 'Accuracy_Val_Bayes','Parametro_SVM','Accuracy_Ent_SVM', 
                     'Accuracy_Val_SVM', 'Profundidad_arbol', 'Accuracy_Ent_arbol','Accuracy_Val_arbol'] )

,Lambda_Reg,Accuracy_Ent_Regresion,Accuracy_Val_Regresion,Variables_Bayes,Accuracy_Ent_Bayes,Accuracy_Val_Bayes,Parametro_SVM,Accuracy_Ent_SVM,Accuracy_Val_SVM,Profundidad_arbol,Accuracy_Ent_arbol,Accuracy_Val_arbol
0,0.1,0.585237,0.594406,"[0, 1, 2, 3, 4, 5]",0.810193,0.797203,0.1,0.808436,0.811189,2,0.799649,0.818182
1,0.5,0.616872,0.538462,"[0, 1, 2, 3, 4, 5, 6]",0.811951,0.783217,1,0.843585,0.839161,3,0.83304,0.832168
2,1,0.671353,0.734266,"[1, 2, 3, 4, 5, 6, 7, 8]",0.796134,0.79021,2,0.843585,0.846154,4,0.838313,0.839161
3,2,0.643234,0.573427,"[0, 1, 3, 5, 6, 8, 9, 10]",0.70123,0.643357,3,0.8471,0.839161,5,0.855888,0.825175
4,3,0.579965,0.58042,"[1, 3, 4, 5, 6, 7, 8, 9]",0.787346,0.804196,4,0.848858,0.839161,6,0.880492,0.79021
5,4,0.690685,0.685315,"[0, 1, 4, 5, 6, 8, 9, 10]",0.766257,0.72028,5,0.850615,0.839161,7,0.892794,0.825175
6,5,0.604569,0.671329,"[0, 1, 2, 4, 6, 7, 8, 10]",0.780316,0.72028,6,0.85413,0.839161,8,0.906854,0.797203
7,10,0.706503,0.622378,"[1, 2, 3, 4, 5, 7, 8, 9, 10]",0.766257,0.755245,7,0.859402,0.832168,9,0.919156,0.818182
8,15,0.659051,0.65035,"[0, 1, 3, 5, 6, 7, 8, 9, 10]",0.697715,0.615385,10,0.871705,0.811189,10,0.926186,0.797203
9,20,0.68717,0.713287,"[1, 2, 3, 5, 6, 7, 8, 9, 10]",0.667838,0.622378,15,0.869947,0.804196,20,0.977153,0.79021


## Con los datos de los modelos, se toman las siguientes conclusiones:

Al resumir todos los modelos en la tabla arriba descrita se puede apreciar cual tiene mayor presición con base a los datos del Accuracy entre el entreno y la validación.

### Regresión
Se toma el modelo que tiene un lambda igual a 1.0 ya que presenta las mejores condiciones.

### Naive Bayes
Se toma el modelo que tiene las variables 1, 3, 4, 5, 6, 7, 8, 9 del modelo, ya que presenta mejores caracteristicas en el modelos.

### SVM
Se toma el modelo que tiene como parametro de elección 2 ya que presenta mejores resultados.

### Árbolo de desición
Se toma el modelo que presenta una profundidad del modelo equivalente a 4 veces.

In [44]:
arbol_validar, _, _ = arbol_segmentacion(X_entreno, X_val, y_entreno, y_val, 4)
y_pred_arbol = arbol_validar.predict(X_val)

svm_validar, _, _ = svm_funcion(X_entreno, X_val, y_entreno, y_val, 2)
y_pred_SVM = svm_validar.predict(X_val)

NB_validar, _, _ = naive_bayes(X_entreno, X_val, y_entreno, y_val, [1, 3, 4, 5, 6, 7, 8, 9])
y_pred_NB = NB_validar.predict(X_val.values[:, [1, 3, 4, 5, 6, 7, 8, 9]])

w_, b_ = regresion(X_entreno, X_val, y_entreno, y_val, 0.001, 1., 1000, '')
y_pred_regre = 1.0*((1 / (1 + np.exp(-(np.matmul(X_val.values,w_)+b_)))) > 0.5)


Epoch: 100, 	 costo = 7.9631
Epoch: 200, 	 costo = 0.8174
Epoch: 300, 	 costo = 0.8108
Epoch: 400, 	 costo = 0.8044
Epoch: 500, 	 costo = 0.7982
Epoch: 600, 	 costo = 0.7921
Epoch: 700, 	 costo = 0.7862
Epoch: 800, 	 costo = 0.7805
Epoch: 900, 	 costo = 0.7750
Epoch: 1000, 	 costo = 0.7696


In [51]:
# y_pred_arbol.shape, y_pred_svm.shape, y_pred_NB.shape, y_pred_regre[:,0]

combinedResults = pd.DataFrame(np.column_stack([y_val, y_pred_arbol, y_pred_SVM, y_pred_NB, y_pred_regre[:,0]]), 
             columns=["YVal", "Àrbol", "SVM", "Naive_Bayes", "Regresión"])

combinedResults.tail(10)

,YVal,Àrbol,SVM,Naive_Bayes,Regresión
133,0.0,0.0,0.0,0.0,0.0
134,0.0,1.0,0.0,0.0,0.0
135,1.0,1.0,1.0,1.0,0.0
136,0.0,0.0,0.0,1.0,1.0
137,0.0,1.0,1.0,1.0,1.0
138,1.0,1.0,1.0,1.0,0.0
139,0.0,0.0,0.0,0.0,0.0
140,0.0,0.0,0.0,0.0,0.0
141,0.0,0.0,0.0,0.0,0.0
142,1.0,1.0,0.0,0.0,0.0


## Resultado Final de modelos evaluados

Se obtendra el resultado de Accuracy para el modelo que trabajamos, a través de una moda para saber si nuestro resultado satisface con lo planteado en el ejercicio, que era tener al menos 80% de satisfacción en sus resultados.

In [53]:
# Obtener la moda de las predicciones de los 4 modelos
from scipy import stats
combinedPred, _ = stats.mode(combinedResults.iloc[:,1:5].values, axis = 1)

# Computar el accuracy combinado
mt.accuracy_score(y_val, combinedPred)

0.8531468531468531

## Definición de Bootstrapping

En pocas palabras consiste en partir de una serie de datos se parte la información para tener pequeños dataset para poder trabajaar los mismos, se puede realizar este ejercicio empleando remplazoas o no, dependiendo el enfoque, al hacer esto tendremos más datos con los cuales podremos obtener estimadores y llegar a tener una conclusión que respalde nuestra información.

Para el caso de este proyecto se podría haber implementado algún ejercicio de Bootstrapping para selecionar más información de validación y generar modelos más robustos.


## K-Fold

Es lo que se conoce como validación cruzada, se emplea para evaluar resultados de un análisis estadistico para determinar y garantizar que son independientes de la partición entre datos de entrenamiento y prueba. Consiste en repetir los ejercicios con otras particiones de datos, se emplea sobre todo para la predicción.

Su nombre proviene de dividir la muestra en k cantidades para poder determinar que el promedio de todos los k valores este dentro de los valores que nosotros esperamos y consideramos adecuados.


# Conclusiones

Trabajar con Feuturing Enginnering es interesante ya que permite reconocer la variables antes de trabajar con las mismas sirve como guia y permite obtener mejores resultados, en el caso de este modelo en particular lo interesante fue que por medio de hacer esto se pudo quitar algunas variables que no eran relevantes para nuestro ejercicio como lo eran el nombre por ejemplo.

Se pudo trabajar también con one-hot encoding para poder cambiar nuestra variables categoricas a dummies para poder clasificar mejor nuestros datos, esto gracias a las facilidades que otorga python en especifico la libreria Sklearn, que resulta bastante completa.

Se trabajo con cuatro modelos distintos entre si que tienen sus ventajas y desvetajas, al permitir trabajar con la libreria Sklear, facilito mucho entender el por que de algunos modelos, ya que en un par de lineas se podía apreciar los resultados, aplicando a estos su respectiva interpretación, mientras que para otros modelos como la regresión implica un conocimiento mayor de programación para llegar a un resultado.

De los modelose en que presento mayor acierto fue el de SVM, que es algo interesante ya que estos modelos trabajan por medio de mínimizar las distancias que tenemos entre los dato para tener la mejor estimación y el modelo que genero los datos con menor certeza fue el de regresión.

El trabajar con encoding que en esta caso consitio en la moda de todos los resultados que obtenido al realizar los ejercicios, encontramos un precisión alta, mayor al 85%, lo que muestra en conclusión que todos nuestros modelos presetan buenos resultados al validarlos y por lo tanto podemos concluir que con un 15% de incerteza nuestro modelo acertara a buenosresultados.

Para trabajos de esta embergadura se podría tomar como guia los cursos en linea que ofrecen algunas universidades en los cuales no se tiene que programar el 100% del código sino ciertas partes fundamentales y lo que se valora más en estos cursos es la interpretación, o trabajar en equipos ya que en mi caso particular al tener un trabajo, el tiempo que tengo para hacer este tipo de proyectos es más corto por lo que podría emplear la calificación o apoyos que se dio en el primer curso que tuvimos de python en el cual se apoyaba en mayor medida.

